## Training

In [1]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

try:
  import tf_agents
except ImportError:
  %pip install tf-agents
  %pip install tf-keras

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(f"gpus: {gpus}")

tf.test.is_built_with_cuda()

gpus: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


True

In [3]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.float_format', lambda x: '%.4f' % x)

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer

from trading_utils import preprocess_data, create_wide_format_data, SYMBOLS
from trading_env import CryptoTradingEnvironment

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Data and Model Paths
DATA_FILEPATH = 'data/ohlcv.csv.gz'
POLICY_SAVE_PATH = 'policy'

# Model Hyperparameters
NUM_TRAINING_STEPS = 1000  # Increased for more meaningful training
NUM_TESTING_STEPS = 500
ALPHA = 1.0 # LinUCB exploration parameter
TEST_SIZE = 0.2
SEED_FUND = 100.0
TRADE_SIZE = 20.0
TRADE_FEE = 0.00025
INVALID_ACTION_PENALTY = -0.00005

TECHNICAL_FEATURES = ['price_change', 'volume_change', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_percent_b', 'bb_bandwidth']
TECHNICAL_FEATURES = ['price_change', 'rsi', 'macd_signal', 'macd_hist', 'bb_percent_b', 'bb_bandwidth']

# Load Data
df = pd.read_csv(DATA_FILEPATH, compression='gzip', parse_dates=['timestamp']) #.set_index('timestamp')
all_data = preprocess_data(df)#.iloc[2000:] # TODO: remove this
observation_df, prices_df = create_wide_format_data(
    all_data, 
    symbols=SYMBOLS, 
    features=TECHNICAL_FEATURES
)

train_observation_df, test_observation_df = train_test_split(
    observation_df, 
    test_size=TEST_SIZE, 
    shuffle=False
)

train_prices_df, test_prices_df = train_test_split(
    prices_df, 
    test_size=TEST_SIZE, 
    shuffle=False
)

scaler = StandardScaler()
train_observation_scaled_df = pd.DataFrame(scaler.fit_transform(train_observation_df), columns=train_observation_df.columns, index=train_observation_df.index)
test_observation_scaled_df = pd.DataFrame(scaler.transform(test_observation_df), columns=train_observation_df.columns, index=test_observation_df.index)


print(f"all_data: {all_data.shape}")
print(f"observation training data: {train_observation_scaled_df.shape}")
print(f"price training data: {train_prices_df.shape}")
print(f"observation testing data: {test_observation_scaled_df.shape}")
print(f"price testing data: {test_prices_df.shape}")

all_data: (111560, 15)
observation training data: (17849, 30)
price training data: (17849, 5)
observation testing data: (4463, 30)
price testing data: (4463, 5)


In [4]:
# Setup Environment
crypto_env = CryptoTradingEnvironment(
    observation_df=train_observation_scaled_df,
    prices_df=train_prices_df,
    symbols=SYMBOLS,
    seed_fund=SEED_FUND,
    trade_size=TRADE_SIZE,
    trade_fee=TRADE_FEE,
    invalid_action_penalty=INVALID_ACTION_PENALTY,
)

tf_env = tf_py_environment.TFPyEnvironment(crypto_env)

# Setup Agent
agent = lin_ucb_agent.LinearUCBAgent(
    time_step_spec=tf_env.time_step_spec(),
    action_spec=tf_env.action_spec(),
    alpha=ALPHA,
    dtype=tf.float32
)

class ShowProgress:
    def __init__(self, total, interval=50):
        self.counter = 0
        self.total = total
        self.interval = interval
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % self.interval == 0:
            reward = np.round(trajectory.reward.numpy()[0], 5)
            print(f"\r{self.counter}/{self.total} Reward: {reward}", end="")

class MetricsObserver:
    def __init__(self, oracle_fn):
        self._oracle_fn = oracle_fn
        self.rewards = []
        self.regrets = []
        self.optimal_rewards = []

    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            reward = trajectory.reward.numpy()[0]
            optimal_reward = self._oracle_fn(trajectory)
            regret = optimal_reward - reward
            
            self.rewards.append(reward)
            self.regrets.append(regret)
            self.optimal_rewards.append(optimal_reward)

    def cum_rewards(self):
        cumulative_reward = np.cumsum(self.rewards)
        cumulative_optimal_reward = np.cumsum(self.optimal_rewards)
        return cumulative_reward, cumulative_optimal_reward

    def cum_averages(self):
        steps = np.arange(len(self.rewards)) + 1
        cumulative_avg_reward = np.cumsum(self.rewards) / steps
        cumulative_avg_optimal_reward = np.cumsum(self.optimal_rewards) / steps
        cumulative_avg_regret = np.cumsum(self.regrets) / steps
        return cumulative_avg_reward, cumulative_avg_optimal_reward, cumulative_avg_regret
        

def train_step(trajectory):
    if not trajectory.is_last():
        time_axised_trajectory = tf.nest.map_structure(lambda t: tf.expand_dims(t, 1), trajectory)
        agent.train(time_axised_trajectory)
        
metrics_observer = MetricsObserver(tf_env.pyenv.envs[0].optimal_reward_oracle)

# 5. Setup Driver
# Set num_steps to the total number of steps you want to train for.
num_steps_to_train = min(len(train_observation_scaled_df) - 1, NUM_TRAINING_STEPS)

driver = dynamic_step_driver.DynamicStepDriver(
    env=tf_env,
    policy=agent.policy,
    num_steps=num_steps_to_train,
    observers=[train_step, metrics_observer, ShowProgress(num_steps_to_train)]
)

print(f"\nStarting training for {num_steps_to_train} steps...")
driver.run()
print("Training finished.")

# Save Policy
print(f"\nSaving the trained policy to: {POLICY_SAVE_PATH}")
saver = policy_saver.PolicySaver(agent.policy)
saver.save(POLICY_SAVE_PATH)
print("Policy saved successfully.")


Starting training for 1000 steps...
Resetting the environment, including the portfolio.
Resetting the environment.


UnboundLocalError: cannot access local variable 'trade_history_amount' where it is not associated with a value

In [ ]:
cumulative_avg_reward, cumulative_avg_optimal_reward, cumulative_avg_regret = metrics_observer.cum_averages()

plt.plot(cumulative_avg_regret)
plt.title('Cumulative Average Regret')
plt.show()

In [ ]:
plt.plot(cumulative_avg_reward, label='Actual')
plt.plot(cumulative_avg_optimal_reward, label='Optimal')
plt.title('Actual vs. Optimal Cumulative Average Reward')
plt.legend()
plt.show()

In [ ]:
cumulative_reward, cumulative_optimal_reward = metrics_observer.cum_rewards()

plt.plot(cumulative_reward, label='Actual')
plt.plot(cumulative_optimal_reward, label='Optimal')
plt.title('Actual vs. Optimal Cumulative Reward')
plt.legend()
plt.show()

In [ ]:
balance_history = tf_env.pyenv.envs[0].balance_history
asset_value_history = tf_env.pyenv.envs[0].asset_value_history
portfolio_value_history = tf_env.pyenv.envs[0].portfolio_value_history

# Plot the portfolio value over time
plt.figure(figsize=(12, 4))
plt.stackplot(train_prices_df.index[:len(balance_history)],[balance_history, asset_value_history], labels=['Cash Balance','Asset Value'])
plt.legend(loc='upper left')
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.grid(True)
plt.show()

In [ ]:
trade_history = tf_env.pyenv.envs[0].trade_history

trade_history_df = pd.DataFrame(trade_history, columns=['Symbol', 'Valid', 'Type', 'Value'])
trade_history_df['Type'].value_counts().plot(kind='barh')

In [ ]:
trade_history_df['Valid'].value_counts().plot(kind='barh')
plt.show()

## Evaluation

In [ ]:
num_steps_to_test = min(len(test_observation_scaled_df) - 1, NUM_TESTING_STEPS)

eval_env = CryptoTradingEnvironment(
    observation_df=test_observation_scaled_df[:num_steps_to_test],
    prices_df=test_prices_df[:num_steps_to_test],
    symbols=SYMBOLS,
    seed_fund=SEED_FUND,
    trade_size=TRADE_SIZE,
    trade_fee=TRADE_FEE,
    invalid_action_penalty=INVALID_ACTION_PENALTY,
)

# 4. Run the Evaluation Loop
print(f"\n--- Starting Evaluation on Test Set ({len(test_observation_scaled_df)} steps) ---")

time_step = eval_env.reset()

counter = 0

# Loop until the environment signals the episode is over
while not time_step.is_last():
    # The policy expects a batch dimension. We create a new TimeStep
    # object with the observation correctly batched.
    counter += 1
    if counter % 50 == 0:
        print(f"\rTraining step {counter}/{num_steps_to_test}", end="")
    
    # A. Add the batch dimension to the observation tensor.
    observation_batch = tf.expand_dims(time_step.observation, 0)
    
    # B. Use the ._replace() method to create a new TimeStep with the batched observation.
    # This is the correct way to modify a TimeStep.
    batched_time_step = time_step._replace(observation=observation_batch)
    
    # C. Get an action from the loaded policy using the batched TimeStep.
    action_step = agent.policy.action(batched_time_step)
    action = action_step.action.numpy()[0]
    
    # D. Step the environment with the original, unbatched action.
    time_step = eval_env.step(action)

print("\n--- Evaluation Finished ---")

In [ ]:
# 5. Report and Plot the Results
balance_history = eval_env.balance_history
asset_value_history = eval_env.asset_value_history
portfolio_value_history = eval_env.portfolio_value_history
seed_fund_value = balance_history[0]
final_portfolio_value = portfolio_value_history[-1]
total_return_pct = ((final_portfolio_value - seed_fund_value) / seed_fund_value) * 100

print(f"Initial Portfolio Value: ${seed_fund_value:,.2f}")
print(f"Final Portfolio Value:   ${final_portfolio_value:,.2f}")
print(f"Total Return on Test Set: {total_return_pct:.2f}%")

# Plot the portfolio value over time
plt.figure(figsize=(12, 4))
plt.stackplot(test_prices_df.index[:len(balance_history)],[balance_history, asset_value_history], labels=['Cash Balance','Asset Value'])
plt.legend(loc='upper left')
plt.title('Agent Performance on Hold-Out Test Data')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.grid(True)
plt.show()